# Shap values 

## Idea central

En cada permutación posible del orden de las variables, se calcula la contribución marginal de cada feature en ese orden. Luego, se promedian esas contribuciones para obtener su valor de Shapley.

## ¿Qué es una contribución marginal?

La contribución marginal de una variable es cuánto cambia la predicción del modelo cuando se agrega esa variable a un conjunto de variables ya conocidas.

Ejemplo con la permutación $[x_2, x_1, x_3]$:

1. Partimos del baseline del modelo, sin variables: f(∅)
2. Agregamos $x_2$: contribución de $x_2 = f({x2}) - f(∅)$
3. Luego agregamos $x_1$: contribución de $x1 = f({x_2, x_1}) - f({x_2})$
4. Finalmente agregamos $x_3$: contribución de $x_3 = f({x_2, x_1, x_3}) - f({x_2, x_1})$

## Repetir para todas las permutaciones posibles

Con 3 variables existen 3! = 6 permutaciones:

1. $[x_1, x_2, x_3]$  
2. $[x_1, x_3, x_2]$  
3. $[x_2, x_1, x_3]$  
4. $[x_2, x_3, x_1]$  
5. $[x_3, x_1, x_2]$  
6. $[x_3, x_2, x_1]$

En cada una de estas permutaciones, calculamos las contribuciones marginales de x1, x2 y x3 en el momento exacto en que se agregan.

## 📊 Cálculo final

Para cada variable, su valor de Shapley se obtiene como el promedio de sus contribuciones marginales a lo largo de todas las permutaciones:

SHAP x1 = promedio de las contribuciones de x1  
SHAP x2 = promedio de las contribuciones de x2  
SHAP x3 = promedio de las contribuciones de x3

## ✅ Predicción explicada

La predicción del modelo para una observación se descompone como:

$f(x_1, x_2, x_3) = f(∅) + \text{SHAP} x_1 + \text{SHAP} x_2 + \text{SHAP} x_3$

Donde:

- `f(∅)` es la predicción promedio del dataset (baseline)
- `SHAP xi` es la contribución de la variable `xi` para esa observación específica

Esta es una explicación completa, justa y basada en teoría de juegos del porqué el modelo predijo lo que predijo para esa observación específica.


In [23]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np

# Dataset simple con 3 variables
X3 = pd.DataFrame({
    'x1': [1, 2, 3, 4],
    'x2': [10, 20, 30, 40],
    'x3': [100, 200, 300, 400]
})
y3 = 3 * X3['x1'] + 2 * X3['x2'] + 0.5 * X3['x3'] + 5

# Entrenamos modelo
model = DecisionTreeRegressor(max_depth=3, random_state=0)
model.fit(X3, y3)

# Observación que queremos explicar
x_exp = pd.DataFrame({'x1': [2], 'x2': [20], 'x3': [200]})

# Baseline (f(∅)): promedio del modelo con datos originales
f_empty = np.mean(model.predict(X3))

# Permutación a analizar: [x2, x1, x3]
# Paso 1: f({}) → baseline

# Paso 2: f({x2}) → fijamos x2 = 20, sampleamos x1 y x3

# Paso 3: f({x2, x1}) → fijamos x2 = 20, x1 = 2, sampleamos x3

# Paso 4: f({x2, x1, x3}) → fijamos x1, x2, x3 (predicción completa)


In [24]:
# Mostramos resultados de esta permutación
print("Permutación: [x2, x1, x3]")
print(f"f(∅) = {round(f_empty, 2)}")
print(f"f({{x2}}) = {round(f_x2, 2)} → Δx2 = {round(delta_x2, 2)}")
print(f"f({{x2, x1}}) = {round(f_x2_x1, 2)} → Δx1 = {round(delta_x1, 2)}")
print(f"f({{x2, x1, x3}}) = {round(f_x2_x1_x3, 2)} → Δx3 = {round(delta_x3, 2)}")


Permutación: [x2, x1, x3]
f(∅) = 187.5
f({x2}) = 132.75 → Δx2 = -54.75
f({x2, x1}) = 132.75 → Δx1 = 0.0
f({x2, x1, x3}) = 78.0 → Δx3 = -54.75


### Código generalizado

In [8]:
from itertools import permutations

# Creamos un nuevo dataset con tres variables
X3 = pd.DataFrame({
    'x1': [1, 2, 3, 4],
    'x2': [10, 20, 30, 40],
    'x3': [100, 200, 300, 400]
})
y3 = 3 * X3['x1'] + 2 * X3['x2'] + 0.5 * X3['x3'] + 5

# Entrenamos un árbol de decisión
model3 = DecisionTreeRegressor(max_depth=3, random_state=0)
model3.fit(X3, y3)

# Observación a explicar
x_exp = pd.DataFrame({'x1': [2], 'x2': [20], 'x3': [200]})
f_full = model3.predict(x_exp)[0]

# Baseline (sin ninguna variable)
f_base = np.mean(model3.predict(X3))

# Definimos todas las permutaciones posibles de orden de entrada
features = ['x1', 'x2', 'x3']
perms = list(permutations(features))

# Calculamos contribución marginal para cada variable en cada orden
shap_values = {'x1': [], 'x2': [], 'x3': []}

for perm in perms:
    known = []
    for i, feat in enumerate(perm):
        # fijamos las variables conocidas hasta ahora
        X_temp = X3.copy()
        for k in known:
            X_temp[k] = x_exp[k].iloc[0]
        f_prev = np.mean(model3.predict(X_temp))
        known.append(feat)
        for k in known:
            X_temp[k] = x_exp[k].iloc[0]
        f_curr = np.mean(model3.predict(X_temp))

        # contribución marginal de esta variable
        delta = f_curr - f_prev
        shap_values[feat].append(delta)

# Promediamos las contribuciones marginales para cada variable
phi = {k: round(np.mean(v), 2) for k, v in shap_values.items()}
phi['baseline'] = round(f_base, 2)
phi['f(x1,x2,x3)'] = round(f_full, 2)
phi['suma total'] = round(f_base + sum(phi[k] for k in features), 2)
phi


{'x1': -9.12,
 'x2': -45.62,
 'x3': 18.25,
 'baseline': 187.5,
 'f(x1,x2,x3)': 151.0,
 'suma total': 151.01}

# Generalización teórica de los valores de Shapley

Los valores de Shapley provienen de la teoría de juegos cooperativos y se utilizan para asignar de manera justa la "recompensa" total (en este caso, la predicción de un modelo) entre todos los jugadores (features), de acuerdo con su contribución individual al resultado.


## Fórmula general

Sea:

- $N = \{1, 2, ..., n\}$ el conjunto total de variables (features)
- $i \in N$ una variable específica
- $S \subseteq N \setminus \{i\}$ un subconjunto de variables que **no contiene** a $i$
- $f(S)$ la predicción del modelo usando solo las variables en $S$
- $f(S \cup \{i\})$ la predicción del modelo al añadir $i$ al conjunto $S$

Entonces, el valor de Shapley para la variable $i$ es:

$$
\phi_i = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|! \cdot (n - |S| - 1)!}{n!} \cdot \left[ f(S \cup \{i\}) - f(S) \right]
$$


## Interpretación de cada componente

- $\phi_i$ = valor de Shapley de la variable $i$
- $f(S \cup \{i\}) - f(S)$ = contribución marginal de $i$ al conjunto $S$
- $\frac{|S|! \cdot (n - |S| - 1)!}{n!}$ = peso que representa la **proporción de permutaciones** donde $i$ es agregada **después de $S$ y antes del resto**

Este peso asegura que todas las posiciones posibles de $i$ en el orden de las variables son consideradas de forma justa.

---

## Descomposición de la predicción

La predicción del modelo para una observación $x$ se puede expresar como:

$$
f(x) = f(\emptyset) + \sum_{i=1}^{n} \phi_i
$$

Donde:

- $f(\emptyset)$ es el valor base o baseline (por ejemplo, la media de todas las predicciones del dataset)
- Cada $\phi_i$ representa la contribución justa de la variable $x_i$ a la predicción $f(x)$


## Propiedades deseables de los valores de Shapley

1. **Eficiencia:**  
   La suma de los valores de Shapley es igual a la diferencia entre la predicción completa y el baseline:
   $$
   \sum_i \phi_i = f(x) - f(\emptyset)
   $$

2. **Simetría:**  
   Si dos variables hacen la misma contribución en todos los contextos, reciben el mismo valor de Shapley.

3. **Ausencia:**  
   Si una variable no cambia nunca la predicción, su Shapley value es 0.

4. **Aditividad:**  
   Si se combinan dos modelos, los Shapley values se suman.


## Conclusión

El valor de Shapley de una variable es el **promedio ponderado de sus contribuciones marginales** al modelo, considerando **todas las combinaciones posibles** en las que puede ser agregada.  
Es una forma matemática rigurosa y justa de entender cómo cada variable contribuye a una predicción individual.
